## Multiplayer Proxy Failure

In [1]:

import module
import numpy as np
import csv
from joblib import Parallel, delayed
from itertools import product

import warnings
warnings.filterwarnings("error")

## Useful functions

Comparatively to the last one: goal is always the same, no inverse, actions are always the same, steps after discard is always the same, all parameters are in the ESN

In [ ]:
def compute_proxy_nodes_from_esn(esn, trials=10):  
    """
    Compute proxy nodes from an Echo State Network (ESN) by running trials with random agent actions.
    The function calculates the average values of the goal node and the proxy nodes based on the ESN's states.
    """

    num_nodes = esn.n
    possible_nodes =  [i for i in range(num_nodes) if i not in esn.action_nodes and i not in esn.goal]
    run_data_base = np.zeros((trials, num_nodes)) 
    avg_goal_values = []
    avg_proxy_values = []
    for k in range(trials):
        # Generate random agent actions
        action_node_values = np.random.choice([-1.0, 0.0, 1.0], size=len(esn.action_nodes))
        
        # Run the ESN with the random agent actions
        states = esn.run( agent_values=action_node_values)
        data_run =  np.mean(states[esn.proxy_discard: esn.proxy_discard + esn.measure_time, :], axis=0)
        data_run[esn.goal] = np.mean(states[esn.goal_discard: esn.goal_discard+ esn.measure_time, esn.goal], axis=0)
        run_data_base[k] = data_run
        # Compute the average values 
        avg_goal_values.append(np.mean(states[:, esn.goal], axis=0))

    correlations = np.mean(np.corrcoef(run_data_base, rowvar= False)[:, esn.goal], axis=1)#here the runs are on axis 0 and the nodes on axis 1

    bin_edges = np.linspace(-1, 1, 100)
    bin_indices =  np.histogram(correlations, bin_edges)[0] 
    bin_indices = [int(k) for k in  bin_indices]
    # check if there are nan values in correlations
    if np.isnan(correlations).any():
        raise ValueError("NaN values found in correlations")
        

    # Sort nodes by correlation in descending order and select top nodes
    sorted_indices = [k for k in np.argsort(correlations)[::-1 ] if k in possible_nodes]
    proxy_nodes = sorted_indices[:1]
    avg_proxy_values.append(np.mean(run_data_base[:, proxy_nodes], axis=0))
    goal_value = np.mean(avg_goal_values)
    proxy_value = np.mean(avg_proxy_values)
    print(bin_indices)

    return proxy_nodes, goal_value, proxy_value, correlations, bin_indices

In [19]:


def optimize_node(esn,  trials, proxy_nodes, is_goal=False):

    n = esn.n 
    num_action_nodes = len(esn.action_nodes)
    start_time = esn.proxy_discard*(not is_goal) + esn.goal_discard*is_goal
    end_time = start_time + esn.measure_time
    # define function to test an agent action 
    def test_agent_action(agent_action, action_value):
        """
        Test the average value of an action node if all other nodes are set to random values
        """
        run_data_base = [0 for i in range(trials)]
        proxy_samples    = np.asarray([])
        goal_samples    = np.asarray([])
        actionVALS = []
        for j in range(trials): 
            action_node_values = np.random.choice([-1.0, 0.0, 1.0], size=len(esn.action_nodes))
            action_node_values[agent_action] = action_value
            # REMOVE actionVALS.append(action_node_values)
            states = esn.run(agent_values=action_node_values)
            proxy_samples = np.concat((proxy_samples,np.mean(states[start_time:end_time, proxy_nodes], axis=0)))
            goal_samples = np.concat((goal_samples,np.mean(states[esn.goal_discard:esn.goal_discard+esn.measure_time, esn.goal], axis=0)))
        try:
            correl_node_goal = np.corrcoef(proxy_samples, goal_samples)[0, 1]
        except RuntimeWarning:
            correl_node_goal = -np.inf
        return np.mean(proxy_samples), np.mean(goal_samples), correl_node_goal
    max_proxy_value = -1
    goal_value_on_max_proxy = -1
    correlation_on_max_proxy = 0
    for i in range(num_action_nodes):
        for val in [-1.0, 1.0]:
            proxy_value, goal_value, correl_node_goal = test_agent_action(i, val)
            if proxy_value > max_proxy_value:
                max_proxy_value = proxy_value
                goal_value_on_max_proxy = goal_value
                correlation_on_max_proxy = correl_node_goal

    return (goal_value_on_max_proxy, max_proxy_value, correlation_on_max_proxy)

In [20]:
def parallel_compute_proxy_failure(param_ESN):
    esn = module.EchoStateNetwork(param_ESN['n'],  spectral_radius=param_ESN['spectral_radius'], alpha = param_ESN['alpha'], 
                                avg_number_of_edges=param_ESN['avg_number_of_edges'], proxy_discard=param_ESN['proxy_discard'],
                                goal_discard=param_ESN['goal_discard'], measure_time=param_ESN['measure_time'])

    proxy_nodes, goal_base, proxy_base, correlations, bin_indices = compute_proxy_nodes_from_esn(esn, trials=param_ESN['trials'])
    correlation_base = np.mean(correlations[proxy_nodes])
    correlation_std = np.std(correlations)
    maxed_goal_value, maxed_proxy_value, correlation_on_max_proxy = optimize_node(esn,  param_ESN['trials'], proxy_nodes)
    optimal_goal_value, optimal_proxy_value, correlation_on_optimal_proxy =  optimize_node(esn,  param_ESN['trials'], esn.goal)
    to_return = {
        'correlation_std': correlation_std,
        'maxed_goal_value': maxed_goal_value,
        'goal_base': goal_base,
        'maxed_proxy_value': maxed_proxy_value,
        'correlation_on_max_proxy': correlation_on_max_proxy,
        'correlation_base': correlation_base,
        'optimal_goal_value': optimal_goal_value,
        'optimal_proxy_value': optimal_proxy_value,
        'correlation_on_optimal_proxy': correlation_on_optimal_proxy,
        'bin_indices': bin_indices,
        'proxy_base': proxy_base
    }
    print("parallel",to_return['bin_indices'])
    return to_return

In [21]:
def parallel_compute_correlation(param_exp,param_ESN):
    esn = module.EchoStateNetwork(param_ESN['n'],  spectral_radius=param_ESN['spectral_radius'], alpha = param_ESN['alpha'], 
                                avg_number_of_edges=param_ESN['avg_number_of_edges'], discard=param_ESN['discard'])
    param_exp['esn'] = esn
    # Compute proxy nodes
    proxy_nodes, goal_base, proxy_base, correlations = compute_proxy_nodes_from_esn(esn, trials=param_ESN['trials'])
    correlations = list(correlations)
    to_return = {
        'correlations': correlations,
    } 
    return to_return

## Main loop to generate experiments

In [46]:
def generate_experimental_data(filename, param_grid, number_of_instances, intention = 'a', skip_to = 0):
    
    # creates a list of the parameters that will vary
    varying_params = [a for a in param_grid.keys() if len(param_grid[a]) > 1]
    # Create a list of all combinations of parameters

    param_combinations = list(product(*param_grid.values()))
    print("param_combinations", len(param_combinations))
    # Create a list of parameter names
    param_names = list(param_grid.keys())
    # Loop through all combinations of parameters

    param_range = range(len(param_combinations))
    if skip_to > 0:
        param_range = range(skip_to, skip_to + 1)
        print("skipping to", skip_to, "out of", len(param_combinations))

    with open(filename,intention) as fd:
        fieldnames = ['correlation_std', 'maxed_goal_value','goal_base','maxed_proxy_value',
                      'correlation_on_max_proxy','correlation_base','optimal_goal_value','optimal_proxy_value','correlation_on_optimal_proxy','bin_indices','proxy_base'] 
        writer = csv.DictWriter(fd, fieldnames=param_names + fieldnames)
        if intention == 'w':
            writer.writeheader()
        for k in param_range:

            params = param_combinations[k]
            # Create a dictionary of parameters
            param_ESN = dict(zip(param_names, params))
            param_ESN['trials'] = 50
            name = 'ESN'+ "".join(['_'+a +'_'+ str(b) for (a,b) in list(zip(param_names, params)) if a in varying_params])
            print("instance", k+1, "out of ", len(param_combinations), "name", name)
            
            delta_goal_values,delta_optimal_goals,delta_proxy_correlations,List_output = [], [], [], []
            # run the parallel computation of proxy nodes
            
            List_output = Parallel(n_jobs=1, return_as='list')(
                [delayed(parallel_compute_proxy_failure)(param_ESN) for _ in range(number_of_instances)]
            )
            print("List_output", List_output)
            for temp_dict in List_output:
                writer.writerow(param_ESN | temp_dict)
            fd.flush()




In [52]:


use_baby_params = False
param_grid = {
    'n': [120, 256, 512, 1024, 2048], 
    'trials': [50],
    'proxy_discard': [50],
    'goal_discard': [50, 150, 250, 450],
    'measure_time': [50],
    'spectral_radius': [1, 1.5, 2, 3],
    'alpha': [0.1],
    "avg_number_of_edges": [1, 2, 4, 8]
}

if use_baby_params:
    param_grid = {
        'n': [40], 
        'trials': [50],
        'proxy_discard': [50],
        'goal_discard': [100],
        'measure_time': [50],
        'spectral_radius': [1.5],
        'alpha': [0.1],
        "avg_number_of_edges": [1]
    }


generate_experimental_data('data_file_dump', param_grid, 1, intention = 'w', skip_to = 27)

param_combinations 320
skipping to 27 out of 320
instance 28 out of  320 name ESN_n_120_goal_discard_150_spectral_radius_2_avg_number_of_edges_8
[37, 30, 41, 69, 26, 68, 90, 17, 73, 77, 33, 79, 16, 80, 70, 54, 73, 56, 68, 28, 49, 42, 22, 63, 36, 21, 52, 57, 70, 46, 29, 54, 17, 74, 18, 20, 13, 25, 20, 59, 31, 80, 23, 40, 70, 24, 38, 55, 39, 61, 81, 27, 24, 34, 55, 57, 83, 43, 55, 19, 63, 63, 66, 14, 39, 36, 57, 72, 26, 50, 44, 80, 77, 46, 87, 25, 17, 84, 29, 42, 52, 76, 26, 26, 44, 67, 10, 46, 68, 81, 74, 83, 29, 47, 17, 18, 24, 85, 79, 74, 81, 27, 26, 61, 21, 12, 64, 40, 54, 77, 67, 32, 20, 80, 53, 71, 13, 52, 30, 99]
parallel [37, 30, 41, 69, 26, 68, 90, 17, 73, 77, 33, 79, 16, 80, 70, 54, 73, 56, 68, 28, 49, 42, 22, 63, 36, 21, 52, 57, 70, 46, 29, 54, 17, 74, 18, 20, 13, 25, 20, 59, 31, 80, 23, 40, 70, 24, 38, 55, 39, 61, 81, 27, 24, 34, 55, 57, 83, 43, 55, 19, 63, 63, 66, 14, 39, 36, 57, 72, 26, 50, 44, 80, 77, 46, 87, 25, 17, 84, 29, 42, 52, 76, 26, 26, 44, 67, 10, 46, 68, 81, 74, 

In [53]:
param_grid = {
    'n': [120, 256, 512, 1024, 2048], 
    'trials': [50],
    'proxy_discard': [50],
    'goal_discard': [50, 150, 250, 450],
    'measure_time': [50],
    'spectral_radius': [1, 1.5, 2, 3],
    'alpha': [0.1],
    "avg_number_of_edges": [1, 2, 4, 8]
}
def reorder_experimental_data(filename, newfilename, param_grid, number_of_instances):
    
    # creates a list of the parameters that will vary
    varying_params = [a for a in param_grid.keys() if len(param_grid[a]) > 1]
    # Create a list of all combinations of parameters

    param_combinations = list(product(*param_grid.values()))
    print("param_combinations", len(param_combinations))
    # Create a list of parameter names
    param_names = list(param_grid.keys())
    # Loop through all combinations of parameters

    fd = open(filename, "r")
    reader = csv.reader(fd, delimiter = ',')
    fw = open(newfilename, "w")
    writer = csv.writer(fw, delimiter=',')
    
    fieldnames = ['correlation_std', 'maxed_goal_value','goal_base','maxed_proxy_value',
                    'correlation_on_max_proxy','correlation_base','optimal_goal_value','optimal_proxy_value','correlation_on_optimal_proxy','proxy_base'] 
        
    counter = 0
    writer.writerow(param_names + fieldnames)
    for line in reader:
        if line != []:
            counter+= 1
            params = param_combinations[counter // number_of_instances]
            del line[9]
            writer.writerow(list(params)+ line )
    fd.close()
    fw.close()
    

        
reorder_experimental_data('data_file',"filtered_data_file.csv", param_grid, 15)

param_combinations 320


In [51]:
import time 

data = np.random.rand(1000000)  # Example correlation data for demonstration
bins = np.linspace(-1, 1, 100)



time_init  = time.time()
bin_nb = np.histogram(data, bins)[0] 
bin_nb= [int(k) for k in bin_nb]
time_end = time.time()
print("Time taken for binning with numpy:", time_end - time_init, "seconds")
print(bin_nb)

Time taken for binning with numpy: 0.006895780563354492 seconds
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10097, 20221, 20259, 20275, 20381, 20295, 19951, 20140, 20222, 20104, 20236, 20302, 20208, 20485, 20181, 20258, 20283, 20058, 20544, 19874, 20143, 20210, 20154, 20051, 20366, 20088, 19957, 20051, 20283, 20144, 20322, 19894, 20217, 20233, 20288, 20056, 20378, 20101, 20218, 20082, 20135, 20364, 20061, 20209, 20182, 20319, 20325, 20334, 20223, 20238]
